In [1]:
import girder_client



apiUrl = "https://imaging.htan.dev/girder/api/v1"

gc = girder_client.GirderClient(apiUrl=apiUrl)
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


{'_accessLevel': 2,
 '_id': '5ebee66965f992910fa19570',
 '_modelType': 'user',
 'admin': True,
 'created': '2020-05-15T18:58:49.562000+00:00',
 'email': 'girder.admin@kitware.com',
 'emailVerified': True,
 'firstName': 'admin',
 'groupInvites': [],
 'groups': [],
 'lastName': 'admin',
 'login': 'admin',
 'otp': False,
 'public': True,
 'size': 168288727353,
 'status': 'enabled'}

In [6]:
## Creating a class for girder interactions that allows me to both cache folders and also if it's not cached
## to create it; most improtantly I am allowing you to specify an entire dierctory structure analogous to
## os.makedirs

class dsaFolderCache:
    def __init__(self, gc,debug=False):
        self.gc = gc
        self.folderCache = {}
        self.collectionCache = {}
        self.debug = debug
        self.cacheHits = 0  ## Generate stats to keep track cache hits
        self.totalLookups = 0
        
    def getFolderId( self, dsaFolderPath ):
        ### This includes both the collection and subsequent folders
        
        if self.debug: print("Looking up %s" % dsaFolderPath)
        ### First see if this is already cached
        if dsaFolderPath in self.folderCache:
            print("CACHED!")
            self.totalLookups +=1
            return self.folderCache[dsaFolderPath]
        else:
            dsaPathParts = dsaFolderPath.split("/")
            ## Make sure all the paths exist..
            for i in range(len(dsaPathParts)):
                subDirPath = "/".join(dsaPathParts[:i+1])
                print(subDirPath)
                self.lookupPathInCollection(subDirPath)
            ## Need to figure out how to return it on the final iteration...
                
    def lookupPathInCollection( self, dsaFolderPath,createIfNotFound=False):
        f = gc.get("/resource/lookup?path=/collection/%s" % dsaFolderPath)
        if f:
            self.folderCache[dsaFolderPath] = f
        else:
            print("Path does not exist",dsaFolderPath)
            ## So I must create it...
            ### Must now try and create the path recursively...

    def loadOrCreateFolder(self, folderName, parentId, parentType, metadata=None):
            """Returns a folder in Girder with the given name under the given
            parent. If none exists yet, it will create it and return it.
            :param folderName: the name of the folder to look up.
            :param parentId: id of parent in Girder
            :param parentType: one of (collection, folder, user)
            :param metadata: JSON metadata string to set on folder.
            :returns: The folder that was found or created.
            """
            children = self.listFolder(parentId, parentType, name=folderName)

            try:
                return next(children)
            except StopIteration:
                return self.createFolder(parentId, folderName, parentType=parentType,
                                         metadata=metadata)            
            
            
## This time should now be using cache
dsaFldr = dsaFolderCache(gc,debug=False)

# dsaFldr.getFolderId("DCC_Bucket_Sync/htan-dcc-htapp/codex_level_1/test-upload/Cyc11_reg1")
# dsaFldr.getFolderId("DCC_Bucket_Sync/htan-dcc-htapp/codex_level_1/test-upload/Cyc11_reg1")

### For now... no begnining slash and no end slash .... need to code this check in

#dsaFldr.getFolderId("DCC_Bucket_Sink/htan-dcc-htapp/codex_level_1/test-upload/Cyc11_reg1")

DCC_Bucket_Sync
DCC_Bucket_Sync/htan-dcc-htapp
DCC_Bucket_Sync/htan-dcc-htapp/codex_level_1
DCC_Bucket_Sync/htan-dcc-htapp/codex_level_1/test-upload
DCC_Bucket_Sync/htan-dcc-htapp/codex_level_1/test-upload/Cyc11_reg1


In [8]:
dsaFldr.getFolderId("DCC_Bucket_Sync/htan-dcc-htapp/codex_level_4/test-upload/Cyc11_reg1")


DCC_Bucket_Sync
DCC_Bucket_Sync/htan-dcc-htapp
DCC_Bucket_Sync/htan-dcc-htapp/codex_level_4
DCC_Bucket_Sync/htan-dcc-htapp/codex_level_4/test-upload


HttpError: HTTP error 400: GET https://imaging.htan.dev/girder/api/v1//resource/lookup?path=/collection/DCC_Bucket_Sync/htan-dcc-htapp/codex_level_4/test-upload
Response text: {"message": "Path not found: collection/DCC_Bucket_Sync/htan-dcc-htapp/codex_level_4/test-upload", "type": "validation"}

In [ ]:
    def _uploadFolderRecursive(self, localFolder, parentId, parentType, leafFoldersAsItems=False,
                               reuseExisting=False, blacklist=None, dryRun=False, reference=None):
        """
        Function to recursively upload a folder and all of its descendants.
        :param localFolder: full path to local folder to be uploaded
        :param parentId: id of parent in Girder, where new folder will be added
        :param parentType: one of (collection, folder, user)
        :param leafFoldersAsItems: whether leaf folders should have all
            files uploaded as single items
        :param reuseExisting: boolean indicating whether to accept an existing item
            of the same name in the same location, or create a new one instead
        :param reference: Option reference to send along with the upload.
        """
        blacklist = blacklist or []
        if leafFoldersAsItems and self._hasOnlyFiles(localFolder):
            if parentType != 'folder':
                raise Exception(
                    ('Attempting to upload a folder as an item under a %s. '
                     % parentType) + 'Items can only be added to folders.')
            else:
                self._uploadFolderAsItem(localFolder, parentId, reuseExisting, dryRun=dryRun)
        else:
            filename = os.path.basename(localFolder)
            if filename in blacklist:
                if dryRun:
                    print('Ignoring file %s as it is blacklisted' % filename)
                return

            print('Creating Folder from %s' % localFolder)
            if dryRun:
                # create a dry run placeholder
                folder = {'_id': 'dryrun'}
            else:
                folder = self.loadOrCreateFolder(
                    os.path.basename(localFolder), parentId, parentType)

            for entry in sorted(os.listdir(localFolder)):
                if entry in blacklist:
                    if dryRun:
                        print('Ignoring file %s as it is blacklisted' % entry)
                    continue
                fullEntry = os.path.join(localFolder, entry)
                if os.path.islink(fullEntry):
                    # os.walk skips symlinks by default
                    print('Skipping file %s as it is a symlink' % entry)
                    continue
                elif os.path.isdir(fullEntry):
                    # At this point we should have an actual folder, so can
                    # pass that as the parent_type
                    self._uploadFolderRecursive(
                        fullEntry, folder['_id'], 'folder', leafFoldersAsItems, reuseExisting,
                        blacklist=blacklist, dryRun=dryRun, reference=reference)
                else:
                    self._uploadAsItem(
                        entry, folder['_id'], fullEntry, reuseExisting, dryRun=dryRun,
                        reference=reference)

            if not dryRun:
                for callback in self._folderUploadCallbacks:
                    callback(folder, localFolder)

In [21]:
m = ['DCC_Bucket_Sync', 'htan-dcc-htapp', 'codex_level_1', 'test_upload', 'Cycll_reg1']


for i in range(len(m)):
    print(i)
    #                 print(dsaPathParts[:i+1],i)
#                 print("/".join(dsaPathParts[:i+1]))
#                 self.lookupPathInCollection("/".join(dsaPathParts[:i+1]))

#             collectionName = dsaPathParts[0]
#             print("Collection name is %s" % collectionName)
            
#             
#             self.folderCache[collectionName] = f

## Add a check to make the sure the path starts with a forward slash
# dsaPath = "DCC_Bucket_Sync/htan-dcc-htapp/codex_level_1/test_upload/Cycll_reg1"   

#dsaFldr.getFolderId("DCC_Bucket_Sync/htan-dcc

0
1
2
3
4


In [24]:
"/".join(m[:1])

'DCC_Bucket_Sync'

In [33]:
pathParts

['DCC_Bucket_Sync',
 'htan-dcc-htapp',
 'codex_level_1',
 'test_upload_Cycll_reg1']